## Package

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import os.path
from pathlib import Path

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import plot_model
from tensorflow.keras import regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D,MaxPool2D
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import glorot_uniform
#import visualkeras
from tensorflow.keras.optimizers import Adam

from IPython.display import Markdown, display
import seaborn as sns

# from lib import proc_img
# from lib import create_gen

def printmd(string):
    #Print with Markdowns    
    display(Markdown(string))

In [2]:
%load_ext autotime
RANDOM_STATE = 42

In [3]:
seed = 7
np.random.seed(seed)

In [4]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(train_images, train_labels, test_size=0.2, shuffle= True)

In [10]:

    def identity_block(X, f, filters, stage, block):

        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        F1, F2, F3 = filters

        X_shortcut = X

        X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

        X = Add()([X, X_shortcut])# SKIP Connection
        X = Activation('relu')(X)

        return X

In [11]:

    def convolutional_block(X, f, filters, stage, block, s=2):

        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'

        F1, F2, F3 = filters

        X_shortcut = X

        X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
        X = Activation('relu')(X)

        X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
        X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

        X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
        X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

        X = Add()([X, X_shortcut])
        X = Activation('relu')(X)

        return X

In [12]:
dense_sizes_1 = [64,128,256,512]
dense_sizes_2 = [64,128,256,512]
learning_rates_1 = [1e-3,1e-1, 1e-2, 1e-4, 1e-5]

s1_filter_sizes_1 = [16,32,64,128,256,512]

s2_filter_sizes_1 = [16,32,64,128,256,512]
s2_filter_sizes_2 = [16,32,64,128,256,512]
s2_filter_sizes_3 = [128,256,512,1024,2048]

s3_filter_sizes_1 = [16,32,64,128,256,512]
s3_filter_sizes_2 = [16,32,64,128,256,512]
s3_filter_sizes_3 = [128,256,512,1024,2048]

s4_filter_sizes_1 = [16,32,64,128,256,512]
s4_filter_sizes_2 = [16,32,64,128,256,512]
s4_filter_sizes_3 = [128,256,512,1024,2048]

s5_filter_sizes_1 = [16,32,64,128,256,512]
s5_filter_sizes_2 = [16,32,64,128,256,512]
s5_filter_sizes_3 = [128,256,512,1024,2048]

dp_sizes_1 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]
dp_sizes_2 = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8]

wd_sizes_1=[0.0,0.1, 0.001, 0.0001]
wd_sizes_2=[0.0,0.1, 0.001, 0.0001]

opt_layer_1=0.001
opt_layer_2=64
opt_layer_3=64
opt_layer_4=16
opt_layer_5=16
opt_layer_6=16
opt_layer_7=128
opt_layer_8=16
opt_layer_9=16
opt_layer_10=128
opt_layer_11=16
opt_layer_12=16
opt_layer_13=128
opt_layer_14=16
opt_layer_15=16
opt_layer_16=128
opt_layer_17=0.0
opt_layer_18=0.0
opt_layer_19=0.0
opt_layer_20=0.0

temp=0
test_acc=0
combinations=0
flag=0
for wd_size_2 in wd_sizes_2 :
    for wd_size_1 in wd_sizes_1 :
        for dp_size_2 in dp_sizes_2 :
            for dp_size_1 in dp_sizes_1 :
                for s5_filter_size_3 in s5_filter_sizes_3:
                    for s5_filter_size_2 in s5_filter_sizes_2:
                        for s5_filter_size_1 in s5_filter_sizes_1:
                            for s4_filter_size_3 in s4_filter_sizes_3:
                                for s4_filter_size_2 in s4_filter_sizes_2:
                                    for s4_filter_size_1 in s4_filter_sizes_1 :
                                        for s3_filter_size_3 in s3_filter_sizes_3:
                                            for s3_filter_size_2 in s3_filter_sizes_2:
                                                for s3_filter_size_1 in s3_filter_sizes_1:
                                                    for s2_filter_size_3 in s2_filter_sizes_3:
                                                        for s2_filter_size_2 in s2_filter_sizes_2:
                                                            for s2_filter_size_1 in s2_filter_sizes_1:
                                                                for s1_filter_size_1 in s1_filter_sizes_1:
                                                                    for dense_size_2 in dense_sizes_2:
                                                                        for dense_size_1 in dense_sizes_1:
                                                                            for learning_rate_1 in learning_rates_1:
                                                                                print(learning_rate_1,dense_size_1,dense_size_2,s1_filter_size_1,s2_filter_size_1,s2_filter_size_2,s2_filter_size_3,s3_filter_size_1,s3_filter_size_2,s3_filter_size_3,s4_filter_size_1,s4_filter_size_2,s4_filter_size_3,s5_filter_size_1,s5_filter_size_2,s5_filter_size_3,dp_size_1,dp_size_2,wd_size_1,wd_size_2)
                                                                                input_shape=(32, 32, 3)
                                                                                X_input = Input(input_shape)

                                                                                X = ZeroPadding2D((1, 1))(X_input)

                                                                                X = Conv2D(s1_filter_size_1, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X = BatchNormalization(axis=3, name='bn_conv1')(X)
                                                                                X = Activation('relu')(X)
                                                                                X = MaxPooling2D((3, 3), strides=(2, 2))(X)

                                                                                X = convolutional_block(X, f=3, filters=[s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='a', s=1)
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='b')
                                                                                X = identity_block(X, 3, [s2_filter_size_1, s2_filter_size_2, s2_filter_size_3], stage=2, block='c')

                                                                                X = convolutional_block(X, f=3, filters=[s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='a', s=2)
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='b')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='c')
                                                                                X = identity_block(X, 3, [s3_filter_size_1, s3_filter_size_2, s3_filter_size_3], stage=3, block='d')

                                                                                X = convolutional_block(X, f=3, filters=[s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='a', s=2)
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='b')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='c')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='d')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='e')
                                                                                X = identity_block(X, 3, [s4_filter_size_1, s4_filter_size_2, s4_filter_size_3], stage=4, block='f')

                                                                                X = convolutional_block(X, f=3, filters=[s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='a', s=2)
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='b')
                                                                                X = identity_block(X, 3, [s5_filter_size_1, s5_filter_size_2, s5_filter_size_3], stage=5, block='c')

                                                                                X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)

                                                                                X= Dropout(dp_size_1)(X)
                                                                                X = Flatten()(X)
                                                                                X=Dense(dense_size_1, activation='relu',kernel_regularizer=regularizers.l2(wd_size_1), name='fc1',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X=Dense(dense_size_2, activation='relu',kernel_regularizer=regularizers.l2(wd_size_2), name='fc2',kernel_initializer=glorot_uniform(seed=0))(X)
                                                                                X= Dropout(dp_size_2)(X)
                                                                                X = Dense(10,activation='softmax', name='fc3',kernel_initializer=glorot_uniform(seed=0))(X)

                                                                                model = Model(inputs=X_input, outputs=X, name='ResNet50')

                                                                                my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]
                                                                                opt = Adam(learning_rate=learning_rate_1)

                                                                                model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                                                                                                                optimizer=opt,
                                                                                                                metrics=['accuracy'])
                                                                                model.fit(x_train,y_train,
                                                                                          batch_size=64,
                                                                                          epochs=100,
                                                                                          validation_data=(x_valid,y_valid),
                                                                                          callbacks=[my_callbacks])
                                                                                

                                                                                _,test_acc = model.evaluate(x_valid, y_valid)
                                                                                print("Current Acc: ",test_acc)
                                                                                print("Best Acc: ",temp)
                                                                                combinations=combinations+1
                                                                                if flag==0 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_1 = learning_rate_1
                                                                                        temp=test_acc
                                                                                    learning_rates_1 = [opt_layer_1]

                                                                                elif flag==1 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_2 = dense_size_1
                                                                                        temp=test_acc
                                                                                    dense_sizes_1 = [opt_layer_2]

                                                                                elif flag==2 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_3 = dense_size_2
                                                                                        temp=test_acc
                                                                                    dense_sizes_2 = [opt_layer_3]

                                                                                elif flag==3 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_4 = s1_filter_size_1
                                                                                        temp=test_acc
                                                                                    s1_filter_sizes_1 = [opt_layer_4]
                                                                                    
                                                                                elif flag==4 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_5 = s2_filter_size_1
                                                                                        temp=test_acc
                                                                                    s2_filter_sizes_1 = [opt_layer_5]
                                                                                    
                                                                                elif flag==5 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_6 = s2_filter_size_2
                                                                                        temp=test_acc
                                                                                    s2_filter_sizes_2 = [opt_layer_6]
                                                                                    
                                                                                elif flag==6 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_7 = s2_filter_size_3
                                                                                        temp=test_acc
                                                                                    s2_filter_sizes_3 = [opt_layer_7]

                                                                                elif flag==7 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_8 = s3_filter_size_1
                                                                                        temp=test_acc
                                                                                    s3_filter_sizes_1 = [opt_layer_8]
                                                                                    
                                                                                elif flag==8 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_9 = s3_filter_size_2
                                                                                        temp=test_acc
                                                                                    s3_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==9 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_10 = s3_filter_size_3
                                                                                        temp=test_acc
                                                                                    s3_filter_sizes_3 = [opt_layer_10]
                                                                                    
                                                                                elif flag==10 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_11 = s4_filter_size_1
                                                                                        temp=test_acc
                                                                                    s4_filter_sizes_1 = [opt_layer_11]
                                                                                    
                                                                                elif flag==11 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_12 = s4_filter_size_2
                                                                                        temp=test_acc
                                                                                    s4_filter_sizes_2 = [opt_layer_9]
                                                                                    
                                                                                elif flag==12 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_13 = s4_filter_size_3
                                                                                        temp=test_acc
                                                                                    s4_filter_sizes_3 = [opt_layer_13]
                                                                                    
                                                                                elif flag==13 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_14 = s5_filter_size_1
                                                                                        temp=test_acc
                                                                                    s5_filter_sizes_1 = [opt_layer_14]
                                                                                    
                                                                                elif flag==14 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_15 = s5_filter_size_2
                                                                                        temp=test_acc
                                                                                    s5_filter_sizes_2 = [opt_layer_15]
                                                                                    
                                                                                elif flag==15 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_16 = s5_filter_size_3
                                                                                        temp=test_acc
                                                                                    s5_filter_sizes_3 = [opt_layer_16]
                                                                                    
                                                                                elif flag==16 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_17 = dp_size_1
                                                                                        temp=test_acc
                                                                                    dp_sizes_1 = [opt_layer_17]
                                                                                    
                                                                                elif flag==17 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_18 = dp_size_2
                                                                                        temp=test_acc
                                                                                    dp_sizes_2 = [opt_layer_18]
                                                                                    
                                                                                elif flag==18 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_19 = wd_size_1
                                                                                        temp=test_acc
                                                                                    wd_sizes_1 = [opt_layer_19]
                                                                                    
                                                                                elif flag==19 :
                                                                                    if test_acc > temp :
                                                                                        opt_layer_20 = wd_size_2
                                                                                        temp=test_acc
                                                                                    wd_sizes_2 = [opt_layer_20]
                                                                            flag=1
                                                                            test_acc=0
                                                                        flag=2
                                                                        test_acc=0
                                                                    flag=3
                                                                    test_acc=0
                                                                flag=4
                                                                test_acc=0
                                                            flag=5
                                                            test_acc=0
                                                        flag=6
                                                        test_acc=0
                                                    flag=7
                                                    test_acc=0
                                                flag=8
                                                test_acc=0
                                            flag=9
                                            test_acc=0
                                        flag=10
                                        test_acc=0  
                                    flag=11
                                    test_acc=0
                                flag=12
                                test_acc=0
                            flag=13
                            test_acc=0
                        flag=14
                        test_acc=0
                    flag=15
                    test_acc=0
                flag=16
                test_acc=0
            flag=17
            test_acc=0
        flag=18
        test_acc=0
    flag=19
    test_acc=0
    
print("######################################################################################################")
print("Trials: ",combinations)
print("Optimal Validation Accuracy",temp)
print("The Final Optimal Values for all block is : ")
print("learning rate: ",opt_layer_1,"\ndense_size_1: ",opt_layer_2,"\ndense_size_2: ",opt_layer_3,"\ns1_filter_size_1: ",opt_layer_4,"\ns2_filter_size_1: ",opt_layer_5,"\ns2_filter_size_2: ",opt_layer_6,"\ns2_filter_size_3: ",opt_layer_7,"\ns3_filter_size_1: ",opt_layer_8,"\ns3_filter_size_2: ",opt_layer_9,"\ns3_filter_size_3: ",opt_layer_10,"\ns4_filter_size_1: ",opt_layer_11,"\ns4_filter_size_2: ",opt_layer_12,"\ns4_filter_size_3: ",opt_layer_13,"\ns5_filter_size_1: ",opt_layer_14,"\ns5_filter_size_2: ",opt_layer_15,"\ns5_filter_size_3: ",opt_layer_16,"\ndp_size_1: ",opt_layer_17,"\ndp_size_2: ",opt_layer_18,"\nwd_size_1: ",opt_layer_19,"\nwd_size_2: ",opt_layer_20)                      

0.001 64 64 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 19s 23ms/step - loss: 2.0650 - accuracy: 0.2122 - val_loss: 2.4501 - val_accuracy: 0.1681
Epoch 2/100
625/625 [==============================] - 14s 23ms/step - loss: 1.7279 - accuracy: 0.3415 - val_loss: 1.7168 - val_accuracy: 0.3618
Epoch 3/100
625/625 [==============================] - 14s 22ms/step - loss: 1.5474 - accuracy: 0.4301 - val_loss: 1.7297 - val_accuracy: 0.3663
Epoch 4/100
625/625 [==============================] - 14s 22ms/step - loss: 1.4471 - accuracy: 0.4710 - val_loss: 1.4326 - val_accuracy: 0.4823
Epoch 5/100
625/625 [==============================] - 14s 23ms/step - loss: 1.3691 - accuracy: 0.5013 - val_loss: 1.5836 - val_accuracy: 0.4440
Epoch 6/100
625/625 [==============================] - 14s 23ms/step - loss: 1.2967 - accuracy: 0.5333 - val_loss: 1.3796 - val_accuracy: 0.5105
Epoch 7/100
625/625 [==============================] - 14s 

Epoch 46/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9191 - accuracy: 0.2776 - val_loss: 1.9360 - val_accuracy: 0.2716
Epoch 47/100
625/625 [==============================] - 13s 21ms/step - loss: 1.9075 - accuracy: 0.2855 - val_loss: 1.9271 - val_accuracy: 0.2737
Epoch 48/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8952 - accuracy: 0.2892 - val_loss: 1.9144 - val_accuracy: 0.2766
Epoch 49/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8845 - accuracy: 0.2925 - val_loss: 1.9040 - val_accuracy: 0.2844
Epoch 50/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8745 - accuracy: 0.2966 - val_loss: 1.8942 - val_accuracy: 0.2916
Epoch 51/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8639 - accuracy: 0.3020 - val_loss: 1.8809 - val_accuracy: 0.2899
Epoch 52/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8518 - accuracy: 0.3051 - val_loss: 1.8710 -

Epoch 1/100
625/625 [==============================] - 16s 22ms/step - loss: 2.0638 - accuracy: 0.2062 - val_loss: 2.1194 - val_accuracy: 0.2374
Epoch 2/100
625/625 [==============================] - 14s 22ms/step - loss: 1.7221 - accuracy: 0.3471 - val_loss: 1.7366 - val_accuracy: 0.3280
Epoch 3/100
625/625 [==============================] - 14s 22ms/step - loss: 1.5461 - accuracy: 0.4256 - val_loss: 1.6678 - val_accuracy: 0.3882
Epoch 4/100
625/625 [==============================] - 13s 21ms/step - loss: 1.4365 - accuracy: 0.4730 - val_loss: 1.8735 - val_accuracy: 0.3521
Epoch 5/100
625/625 [==============================] - 13s 22ms/step - loss: 1.3497 - accuracy: 0.5129 - val_loss: 1.5709 - val_accuracy: 0.4379
Epoch 6/100
625/625 [==============================] - 13s 21ms/step - loss: 1.2766 - accuracy: 0.5406 - val_loss: 1.5274 - val_accuracy: 0.4543
Epoch 7/100
625/625 [==============================] - 14s 22ms/step - loss: 1.2096 - accuracy: 0.5664 - val_loss: 1.8784 - val_ac

313/313 [==============================] - 3s 9ms/step - loss: 1.3111 - accuracy: 0.5832
Current Acc:  0.5831999778747559
Best Acc:  0.6079999804496765
0.001 256 256 16 16 16 128 16 16 128 16 16 128 16 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 18s 25ms/step - loss: 2.0502 - accuracy: 0.2219 - val_loss: 1.9462 - val_accuracy: 0.2724
Epoch 2/100
625/625 [==============================] - 15s 23ms/step - loss: 1.6878 - accuracy: 0.3715 - val_loss: 2.3528 - val_accuracy: 0.2711
Epoch 3/100
625/625 [==============================] - 14s 23ms/step - loss: 1.5390 - accuracy: 0.4369 - val_loss: 1.6888 - val_accuracy: 0.3947
Epoch 4/100
625/625 [==============================] - 15s 24ms/step - loss: 1.4342 - accuracy: 0.4807 - val_loss: 1.5733 - val_accuracy: 0.4559
Epoch 5/100
625/625 [==============================] - 14s 23ms/step - loss: 1.3586 - accuracy: 0.5112 - val_loss: 1.4806 - val_accuracy: 0.4803
Epoch 6/100
625/625 [=============================

625/625 [==============================] - 13s 21ms/step - loss: 1.4852 - accuracy: 0.4536 - val_loss: 1.5180 - val_accuracy: 0.4519
Epoch 4/100
625/625 [==============================] - 14s 22ms/step - loss: 1.3709 - accuracy: 0.5070 - val_loss: 1.4641 - val_accuracy: 0.4797
Epoch 5/100
625/625 [==============================] - 13s 21ms/step - loss: 1.2600 - accuracy: 0.5500 - val_loss: 1.7813 - val_accuracy: 0.4398
Epoch 6/100
625/625 [==============================] - 14s 22ms/step - loss: 1.1687 - accuracy: 0.5811 - val_loss: 1.4644 - val_accuracy: 0.5121
Epoch 7/100
625/625 [==============================] - 13s 22ms/step - loss: 1.0885 - accuracy: 0.6144 - val_loss: 1.3206 - val_accuracy: 0.5345
Epoch 8/100
625/625 [==============================] - 14s 22ms/step - loss: 1.0238 - accuracy: 0.6393 - val_loss: 1.8419 - val_accuracy: 0.4248
Epoch 9/100
625/625 [==============================] - 14s 22ms/step - loss: 0.9624 - accuracy: 0.6606 - val_loss: 1.2468 - val_accuracy: 0.56

...

625/625 [==============================] - 13s 21ms/step - loss: 0.3716 - accuracy: 0.8722 - val_loss: 1.0083 - val_accuracy: 0.6946
Epoch 19/100
625/625 [==============================] - 14s 22ms/step - loss: 0.3354 - accuracy: 0.8838 - val_loss: 1.1558 - val_accuracy: 0.6671
Epoch 20/100
625/625 [==============================] - 14s 22ms/step - loss: 0.3059 - accuracy: 0.8935 - val_loss: 1.1271 - val_accuracy: 0.6858
Epoch 21/100
313/313 [==============================] - 3s 8ms/step - loss: 1.2437 - accuracy: 0.6559
Current Acc:  0.6559000015258789
Best Acc:  0.670199990272522
0.001 256 64 512 16 16 128 16 256 128 256 256 128 256 16 128 0.0 0.0 0.0 0.0
Epoch 1/100
625/625 [==============================] - 16s 22ms/step - loss: 1.8196 - accuracy: 0.3193 - val_loss: 2.2323 - val_accuracy: 0.2603
Epoch 2/100
625/625 [==============================] - 13s 21ms/step - loss: 1.4860 - accuracy: 0.4579 - val_loss: 1.8462 - val_accuracy: 0.3730
Epoch 3/100
625/625 [=======================

625/625 [==============================] - 17s 23ms/step - loss: 1.8993 - accuracy: 0.2849 - val_loss: 2.1285 - val_accuracy: 0.2899
Epoch 2/100
625/625 [==============================] - 15s 23ms/step - loss: 1.5376 - accuracy: 0.4361 - val_loss: 1.7602 - val_accuracy: 0.3876
Epoch 3/100
625/625 [==============================] - 14s 23ms/step - loss: 1.3691 - accuracy: 0.5074 - val_loss: 1.4494 - val_accuracy: 0.4958
Epoch 4/100
625/625 [==============================] - 14s 23ms/step - loss: 1.2420 - accuracy: 0.5561 - val_loss: 1.8792 - val_accuracy: 0.4208
Epoch 5/100
625/625 [==============================] - 13s 21ms/step - loss: 1.1260 - accuracy: 0.6037 - val_loss: 1.7940 - val_accuracy: 0.4359
Epoch 6/100
625/625 [==============================] - 15s 24ms/step - loss: 1.0445 - accuracy: 0.6348 - val_loss: 1.3266 - val_accuracy: 0.5541
Epoch 7/100
625/625 [==============================] - 14s 22ms/step - loss: 0.9619 - accuracy: 0.6630 - val_loss: 1.4798 - val_accuracy: 0.49

Epoch 6/100
625/625 [==============================] - 13s 21ms/step - loss: 1.1654 - accuracy: 0.5936 - val_loss: 1.7527 - val_accuracy: 0.4498
Epoch 7/100
625/625 [==============================] - 13s 21ms/step - loss: 1.0960 - accuracy: 0.6211 - val_loss: 1.2348 - val_accuracy: 0.5671
Epoch 8/100
625/625 [==============================] - 14s 22ms/step - loss: 1.0200 - accuracy: 0.6489 - val_loss: 1.4233 - val_accuracy: 0.5302
Epoch 9/100
625/625 [==============================] - 13s 21ms/step - loss: 0.9584 - accuracy: 0.6714 - val_loss: 1.5651 - val_accuracy: 0.5042
Epoch 10/100
625/625 [==============================] - 13s 21ms/step - loss: 0.9010 - accuracy: 0.6922 - val_loss: 1.1959 - val_accuracy: 0.6006
Epoch 11/100
625/625 [==============================] - 14s 22ms/step - loss: 0.8452 - accuracy: 0.7123 - val_loss: 1.2108 - val_accuracy: 0.5950
Epoch 12/100
625/625 [==============================] - 13s 21ms/step - loss: 0.7971 - accuracy: 0.7284 - val_loss: 1.3002 - val

625/625 [==============================] - 13s 21ms/step - loss: 1.5052 - accuracy: 0.4593 - val_loss: 1.4748 - val_accuracy: 0.4772
Epoch 4/100
625/625 [==============================] - 13s 21ms/step - loss: 1.3801 - accuracy: 0.5095 - val_loss: 1.7481 - val_accuracy: 0.3900
Epoch 5/100
625/625 [==============================] - 14s 22ms/step - loss: 1.2846 - accuracy: 0.5526 - val_loss: 1.4749 - val_accuracy: 0.4930
Epoch 6/100
625/625 [==============================] - 14s 22ms/step - loss: 1.2062 - accuracy: 0.5814 - val_loss: 1.2764 - val_accuracy: 0.5650
Epoch 7/100
625/625 [==============================] - 13s 21ms/step - loss: 1.1243 - accuracy: 0.6132 - val_loss: 1.4041 - val_accuracy: 0.5377
Epoch 8/100
625/625 [==============================] - 13s 21ms/step - loss: 1.1013 - accuracy: 0.6208 - val_loss: 1.2514 - val_accuracy: 0.5635
Epoch 9/100
625/625 [==============================] - 14s 22ms/step - loss: 0.9972 - accuracy: 0.6581 - val_loss: 1.2467 - val_accuracy: 0.59

Epoch 1/100
625/625 [==============================] - 17s 23ms/step - loss: 2.1788 - accuracy: 0.1477 - val_loss: 1.9279 - val_accuracy: 0.2045
Epoch 2/100
625/625 [==============================] - 14s 22ms/step - loss: 2.0079 - accuracy: 0.1920 - val_loss: 2.7509 - val_accuracy: 0.1976
Epoch 3/100
625/625 [==============================] - 14s 22ms/step - loss: 1.9418 - accuracy: 0.2278 - val_loss: 1.8918 - val_accuracy: 0.2418
Epoch 4/100
625/625 [==============================] - 14s 22ms/step - loss: 1.8935 - accuracy: 0.2447 - val_loss: 1.8874 - val_accuracy: 0.2593
Epoch 5/100
625/625 [==============================] - 13s 21ms/step - loss: 1.8389 - accuracy: 0.2705 - val_loss: 1.8655 - val_accuracy: 0.2834
Epoch 6/100
625/625 [==============================] - 14s 22ms/step - loss: 1.7373 - accuracy: 0.3257 - val_loss: 1.8164 - val_accuracy: 0.3471
Epoch 7/100
625/625 [==============================] - 13s 21ms/step - loss: 1.6176 - accuracy: 0.3819 - val_loss: 1.6115 - val_ac